**Load Data**

**Import Library**

In [ ]:
#Data analysis   
import pandas as pd 
import numpy as np
import csv
import nltk
import tensorflow as tf

#Data visualisation 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
from matplotlib import ticker
%config InlineBackend.figure_format = 'svg' 

In [ ]:
import pandas as pd

df = pd.read_csv("SINGGALANG.tsv", sep='\t')
print(df)

In [ ]:
list_kalimat = []
nomor_kalimat = 1

#Open file in read mode
with open('/content/SINGGALANG.tsv', encoding='UTF-8') as read_obj:
    # Pass the file object to reader() to get the reader object
    csv_reader = csv.reader(read_obj, delimiter='\t', quotechar=None) #https://www.petanikode.com/python-csv/ 
#https://docs.python.org/id/3.9/library/csv.html
#Menginstruksikan reader untuk tidak melakukan pemrosesan khusus terhadap karakter kutipan
    #Iterate over each row in the csv using reader object
    for row in csv_reader:
        #Row variable is a list that represents a row in csv
        if len(row) == 0:
            nomor_kalimat += 1 
        else:
            tmp_row= row
            tmp_row.append(nomor_kalimat)
            list_kalimat.append(tmp_row)

dataset = pd.DataFrame(list_kalimat, columns=['token', 'entitas bernama', 'kalimat'])

In [ ]:
dataset.head(10)

In [ ]:
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
#Kelas untuk mengambil kalimat dari dataset
class getsentence(object): #deklarasikan class dengan nama getsentence
    
    def __init__(self, data): #Deklarasikan “variabel” menggunakan kata “self” yang menunjukkan bahwa “variabel” merupakan milik dari class “getsentence”
      self.n_sent = 1.0
      self.dataset = dataset
      self.empty = False 
      
      #Lambda adalah fungsi untuk mengembalikan nilai 
      agg_func = lambda s: [(t, e) for t, e in zip(s["token"].values.tolist(),
                                                 s["entitas bernama"].values.tolist())]                                           
      self.grouped = self.dataset.groupby("kalimat").apply(agg_func)
      self.sentences = [s for s in self.grouped]

In [ ]:
getter = getsentence(dataset)
kalimat = getter.sentences

#Tampilan kalimat adalah sebagai berikut 
print(kalimat[0]) #Untuk kalimat pertama

In [ ]:
longest_sen = max(kalimat, key=len)
longest_sen_len = len(longest_sen)

In [ ]:
print (longest_sen_len)

In [ ]:
print (longest_sen)

In [ ]:
shortest_sen = min(kalimat, key=len)
shortest_sen_len = len(shortest_sen)

In [ ]:
print (shortest_sen_len)

In [ ]:
print (shortest_sen)

In [ ]:
print("Jumlah kalimat: ", len(dataset.groupby(['kalimat'])))
words = list(set(dataset["token"].values))
n_words = len(words)
print("Jumlah kata unik : ", n_words)
tags = list(set(dataset["entitas bernama"].values))
print("Entitas bernama:", tags)
n_tags = len(tags)
print("Jumlah entitas bernama: ", n_tags)

In [ ]:
#Words tagged as Place
dataset.loc[dataset['entitas bernama'] == 'Place', 'token'].head()

In [ ]:
dataset.iloc[220085] 

In [ ]:
#Words tagged as Person
dataset.loc[dataset['entitas bernama'] == 'Person', 'token'].head()

In [ ]:
#Words tagged as Organization
dataset.loc[dataset['entitas bernama'] == 'Organisation', 'token'].head()

In [ ]:
#Words tagged as O
dataset.loc[dataset['entitas bernama'] == 'O', 'token'].head()

In [ ]:
dataframe = dataset.groupby("entitas bernama" )
dataframe["entitas bernama"].count()

In [ ]:
ncount = len(dataset)

plt.figure(figsize=(10.5, 8))
ax = sns.countplot(x = "entitas bernama", data = dataset)
plt.title('Distribusi Frekuensi Entitas Bernama')
plt.xlabel('Entitas Bernama')

# Buat twin axis
ax2 = ax.twinx()

# Tukar sehingga count axis di kanan, frequency di kiri
ax2.yaxis.tick_left()
ax.yaxis.tick_right()

# Tukar juga label
ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')

ax.set_ylabel('Jumlah')
ax2.set_ylabel('Frekuensi [%]')

for p in ax.patches:
    x = p.get_bbox().get_points()[:,0]
    y = p.get_bbox().get_points()[1, 1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
                ha = 'center', va = 'bottom') # atur alignment teks

# Gunakan LinearLocator untuk memastikan jumlah tick yang benar
ax.yaxis.set_major_locator(ticker.LinearLocator(11))

# Perbaiki rentang frekuensi menjadi 0-100
ax2.set_ylim(0, 100)
ax.set_ylim(0, ncount)

# Gunakan MultipleLocator untuk memastikan spaci antar-tick yaitu 10
ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))

# Atur sehingga grid pada ax2 off, supaya gridlines tidak ditampilkan
# pada bagian atas bar
ax2.grid(None)

In [ ]:
ncount = len(dataset.loc[dataset['entitas bernama'] != 'O'])

plt.figure(figsize=(10.5, 8))
ax = sns.countplot(x = "entitas bernama", data = dataset.loc[dataset['entitas bernama'] != 'O'])
plt.title('Distribusi Frekuensi Entitas Bernama Selain \'O\'')
plt.xlabel('Entitas Bernama')

# Buat twin axis
ax2 = ax.twinx()

# Tukar sehingga count axis di kanan, frequency di kiri
ax2.yaxis.tick_left()
ax.yaxis.tick_right()

# Tukar juga label
ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')

ax.set_ylabel('Jumlah')
ax2.set_ylabel('Frekuensi [%]')

for p in ax.patches:
    x = p.get_bbox().get_points()[:,0]
    y = p.get_bbox().get_points()[1, 1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
                ha = 'center', va = 'bottom') # atur alignment teks

# Gunakan LinearLocator untuk memastikan jumlah tick yang benar
ax.yaxis.set_major_locator(ticker.LinearLocator(11))

# Perbaiki rentang frekuensi menjadi 0-100
ax2.set_ylim(0, 100)
ax.set_ylim(0, ncount)

# Gunakan MultipleLocator untuk memastikan spaci antar-tick yaitu 10
ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))

# Atur sehingga grid pada ax2 off, supaya gridlines tidak ditampilkan
# pada bagian atas bar
ax2.grid(None)

In [ ]:
# Menampilkan 20 token pertama yang paling banyak muncul 
dataset['token'].value_counts()[:20]

In [ ]:
# Visualisasi data 20 token yang paling banyak muncul
dataset['token'].value_counts()[:20].plot(kind='barh')

In [ ]:
 #melihat url yang muncul pada dataset
 import re 
  
def Find(string): 
  
    # findall() has been used  
    # with valid conditions for urls in string 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url] 
      
# Driver Code 
dataset_url = open('SINGGALANG.tsv').read()
print("Urls: ", Find(dataset_url)) 

In [ ]:
 # Menghitung jumlah punctuation
dataset_punct = open('SINGGALANG.tsv').read()

dataset_punct  = dataset_punct.replace("--", " ")

for symbol in "-'":
   dataset_punct  = dataset_punct.replace(symbol + " ", "")
   dataset_punct  = dataset_punct.replace(" " + symbol, "")

for symbol in ".,/'-":
    print (symbol, dataset_punct.count(symbol)) 

##**3. Text Preprocessing** 

*   Expand Contraction

In [ ]:
dataset_prep = pd.DataFrame(list_kalimat, columns=['token', 'entitas bernama', 'kalimat'])

In [ ]:
# Memperbaiki format penulisan pada kolom token
# re.sub = setiap string yang sesuai dengan pola akan digantikan dengan string yang kita inginkan.
# re.search = searches for that pattern within the string.
def perbaikan_format_tanda_baca(dataset_prep):    
    for i in dataset_prep.index:
        token = re.sub('([.,!?()])', r'\1 ', dataset_prep.at[i, 'token'])
        dataset_prep.at[i, 'token'] = re.sub('\s{2,}', ' ', token)
    return dataset_prep
perbaikan_format_tanda_baca(dataset_prep)

In [ ]:
dataset_prep.iloc[24001]

In [ ]:
# Dictionary yang digunakan
import re
contractions_dict = {
"a.l. ": "antara lain",
"A.M.v.B. ": "Algemene Maatregel van Bestuur",
"a.n. ": "atas nama",
"adm. ": "administrasi",
"UU " : "undang-undang",
"art. ": "artikel",
"Ass. Supt. ":"asisten superintenden",
"B.A. ": "Bachelor of Arts",
"B.B.A. ": "Bachelor of Business Administration",
"B.Ch.E ": "Bachelor of Chemical Engineering",
"B.Sc. ": "Bachelor of Science",
"Bpk. ": "Bapak",
"BRAy. ": "Bendara Raden Ayu",
"CC ": "carbon copy",
"D.Sc. ": "doctor of science",
"dkk. ": "dan kawan-kawan",
"dll. ": "dan lain-lain",
"Dr.h.c ": "Doctor Honoris Causa",
"Dr.Phil. ": "Doctor of Philosophy",
"Dr.Th. ": "Doctor Theologiae",
"dr. ": "dokter medis (profesi)",
"Dr. ": "Doktor (gelar akademik)",
"Dra. ": "Doktoranda",
"Drg. ": "Dokter Gigi",
"drh. ": "dokter hewan",
"Drs. ": "Doktorandus",
"dsb. ": "dan lain sebagainya", 
"dsb. " : "dan sebagainya",
"dst. ": "dan seterusnya",
"E.Z. ": "Economische Zaken",
"ed. ": "editor", 
"ed. ": "edisi",
"faks. " : "faksimile",
"fol. " : "folio",
"Fr. " : "frater",
"Gg. " : "gang",
"gg. " : "gang",
"H. " : "Haji",
"Hj. " : "Hajah",
"hlm. " : "halaman",
"Ir. " : "insinyur",
"jln. " :"jalan ",
"jl. " : "jalan ",
"Jl. " : "jalan ",
"jo. " : "juncto",
"Jr. " : "junior",
"K.H. " : "kiai haji",
"K. " : "kiai",
"ka. " : "Kepala",
"kab. " : "kabinet", 
"kab. " : "kabupaten",
"kapt. " : "kapten",
"kec. " : "kecamatan",
"kel. " : "kelurahan",  
"kel. " : "keluarga",
"kep. " : "keputusan",
"kp. " : "Kampung",
"lamp. " : "lampiran",
"LL.B " : "Legum Baccalaureus",
"LL.D " : "Legum Doctor",
"log. " : "logistik",
"M.A. " :"master of arts",
"M.Ag. " : "magister agama",
"M.B.A. " : "master of business administration",
"M.Hum. " : "Magister Humaniora",
"M.Hum. " : "magister humaniora",
"M.Kes. " : "magister kesehatan",
"M.Kom. " : "magister komunikasi",
"M.M. " : "magister manajemen",
"M.P.A. " : "master of public administration",
"M.P.H. " : "master of public health",
"M.P. " : "magister pertanian",
"M.Pd. " : "magister pendidikan",
"M.Ph. " : "master of philosophy",
"M.Sc. " : "master of science",
"M.Si. " : "magister sains",
"M.Sn. " : "magister seni",
"M.T. " : "magister teknik",
"Mgr. " : "monseignor",
"Mr. " : "meester in de rechten",
"Mrs. " : "Mistress",
"Nb " : "Notabene",
"Nn. " : "Nona",
"no. " : "nomor",
"No. " : "nomor",
"Ny. " : "nyonya",
"op cit. " : "opere citato",
"org. " : "Orang",
"p.p. " : "pulang pergi",
"Ph.D. " : "philosophiae doctor",
"pjs. " : "Pejabat sementara",
"PPn. " : "pajak pendapatan",
"PPN " : "pajak pertambahan nilai",
"prof. " : "profesor",
"Psi. " : "Psikologi",
"psw. " : "Pesawat (telepon)",
"R.A. " : "Raden Ajeng",
"red. " : "redaksi",
"reg. " : "register",
"rhs. " : "Rahasia",
"Rr. " : "Raden roro",
"S.Ag. ": "Sarjana Agama",
"S.E. ": "Sarjana Ekonomi",
"S.H. ": "Sarjana Hukum",
"S.Hut. ": "Sarjana Kehutanan",
"S.I.Kom. ": "Sarjana Ilmu Komunikasi",
"S.K.M. ": "Sarjana Kesehatan Masyarakat",
"S.Kedg. ": "Sarjana Kedokteran Gigi",
"S.Kedh. ": "Sarjana Kedokteran Hewan",
"S.Kom. ": "Sarjana Ilmu Komputer",
"S.Pd. ": "Sarjana Pendidikan",
"S.Pd ": "Sarjana Pendidikan",
"S.Pol. ": "Sarjana IImu Politik",
"S.Psi. ": "Sarjana Psikologi",
"S.S. ": "Sarjana Sastra",
"S.Si. ": "Sarjana Sains",
"S.Sn. ": "Sarjana Seni",
"S.Sos. ": "Sarjana Ilmu Sosial",
"S.T. ": "Sarjana Teknik",
"S.Tekp. " : "Sarjana Teknologi Pertanian",
"S.Th. ": "Sarjana Theologi",
"sbb. ": "sebagai berikut",
"ssk. ": "satuan saham kolektif", 
"ssk. ":"satuan setingkat kompi",
"St. ": "Sutan",
"SY. ": "Serikat Yesuit",
"T. " : "teuku",
"Tap. ": "Ketetapan",
"Tb. ": "Tubagus",
"tel. ":" telepon",
"vol. ": "volume",
"ybs. ": "yang bersangkutan",
"ytc. ": "Yang Tercinta",
"yth. ": "Yang Terhormat",
}